In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import pandas as pd
import numpy as np

# Boosting your score (hopefully?)
After running a standar Random Forrest Regressor *(and getting a submission worthy of a 905th spot)*, it seems like we are going to have to pull out some heavier artillery to crack this one. Now, mind you I did little to no tinkering with the setup of the regressor, so that is something that I really to explore before moving on to the next thing. 

After doing a bit of reasearch I discovered that what I want to do is called **"Hyperparameter Tunning"**. This sounds **hyper**-complicated (haha), so I want to take some time to dive deeper explain what it means.

# 1. What is a Hyperparameter?
Think about these **hyperparameters** as the *factory settings* of your machine learning model. Out of the box, you throw some data into the model and it will do its best at giving you a prediction using its standard settings. You don't initially assign these parameters any value, but you can at any point if desired. 

**Hyperparameter Tunning** is the process of tinkering with and modifying these settings with the goal of getting more accurate predictions out of your model.

![Mechanic](https://media.giphy.com/media/PqfKAMBfcqwVodnDKs/giphy.gif)
<cite> <center> Tightening the bolts of your own ML Robot </center><cite>

The thing is, you don't really know what parameters will work best until you start trying different combinations. So to find the best setup you are going to have to get your hands dirty and do some ol'fashioned **trial-and-error** work. However, you must be wary of the most common issues in machine learning: **overfitting**

# 2. Overfitting and Corss Validation ☯

You will usually be dealing with two sepparate datasets when you are applying your models, a training and test data set. As the names imply, you use your training data set to teach your model how to make predictions (to train it) and your test data set to evaluate how good is your model at making predictions. 

A common issue is that your model will be *extremelly* good at making predictions on your training data, but performs really poorly on your test data. This is known as **overfitting**. If we tune and optimize our model for our training data, it will be useless when we try to apply it to new data points. 

Here is where we introduce **Cross Validation**! In this notebook we will be using the most common method, <u>K-Fold</u>. 

## K-Fold Cross Validation

Like I mentioned earlier, you usually split your data set into train and test whenever you are facing a machine learning problem. 

**With K-Fold CV we go a step further, we split our training data into K number of subsets (or Folds). We then itratively fit our model K number of times, training it on K-1 folds and validation on the Kth fold.**

This can be a bit confusing to a beginner like me, so here is a graphic to better explain:

![K-FoldCV](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)


So each time we train the model, we use different parameter settings. At the end:
* We compare all of the models performances 
* Select the best one (along with its finelly tunned parameters 😉) 
* Train the whole data set using those settings

Even though this sounds like a very complicated and tedious process, the Scikit-Learn library makes it really easy and straightforward!

# 3. Getting our Hands Dirty
## What are our Parameters? 
Let's first take a look at what parameter we are dealing with in our RandomForrestRegressor model:

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(random_state = 0)
print('Parameters used:\n')
print(regressor.get_params())

**Way too many parameters**. I don't know what half of these mean to be honest. This is where documentation comes in handy! If we look at the random forest documentation, we can find definitions for all of these parameters. For now I will only be looking at:

* n_estimators: The number of trees in the forest.
* max_features: The number of features to consider when looking for the best split
* max_depth: The maximum depth of the tree
* min_samples_split: The minimum number of samples required to split an internal node
* min_samples_leaf: The minimum number of samples required to be at a leaf node
* bootstrap: Whether bootstrap samples are used when building trees

If you would like a more in-depth explanation of these parameters take a look at the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

## Random Sampling
We will be using *RandomizedSearchCV* to narrow dowm what values will work the best for our parameters. We will first create a **"Grid"** of values to assing to our paramters:

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

Now, we will use our module to pick different combinations of paramters to cross-validate our model with. For this analysis we will using 100 different parameter combinations and 3 folds:

In [ ]:
# Data
train = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')

# Dropping id column
train.drop(['id'], axis=1, inplace= True)
test.drop(['id'], axis=1, inplace= True)

# Sepparating our matrix of features and prediction vector
X = train.iloc[:, 0:-1].values
y = train.iloc[:, -1].values

# Training and Testing Sets
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.3, random_state = 0)

The following code block would be the next step in finding the best paramter settings:

<code>from sklearn.model_selection import RandomizedSearchCV
regressor_random = RandomizedSearchCV(estimator= regressor, param_distributions= random_grid, n_iter=10, cv=2, verbose=2, random_state=0, n_jobs=-1)
regressor_random.fit(X_train,y_train)</code>

<div class="alert alert-block alert-info"> 📌 This seems like its going to be way more complicated than I expected it to be. Given the large size of the data, and the large number of parameters, the training time is going to be insanely long. </div>

# 4. Learned My Lesson

I thought that this approach would be the next logical step at increasing my score. It seems like my hypothesis was wrong. The processing time needed to do a randomized search with this magnitude of data is too long, therefore this approach doesn't seem to be the most optimal. 

I have done some more research and it seems like there are many things I could have done better in this approach, [here is a good forum post about it](https://stats.stackexchange.com/questions/270315/how-to-speed-up-hyperparameter-optimization). I will be trying some of these tips in a future analysis as well as doing some research into [Bayesian Optimization](https://machinelearningmastery.com/what-is-bayesian-optimization/)

# Thank you

As many have said before, data science is a lot of trial and error to find the best solutions. Even though it can be discouraging at times, failing is a great opportunity to learn new things. This is a prime example. Thank you for taking the time to read this notebook, and as always please let me know if you ahve any insight or tips on this topic.

Best,
Jose